# Data Proprocessing

In [32]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

import os, sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))


In [2]:
# Read the dataset
df = pd.read_csv('../data/cleaned_data.csv', low_memory=False, index_col=False)

Understand the data

In [3]:
# Explore the first five rows
df.head(5)

,Unnamed: 0,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,54.824561,0.0
4,4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


**Identify Unique Categories:**

Check unique values for categorical features and decide on encoding strategy.

In [4]:
# Get categorical columns
categorical_columns = df.select_dtypes(include='object').columns

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Column': categorical_columns,
    'DataType': [df[col].dtype for col in categorical_columns],
    'NumUniqueValues': [df[col].nunique() for col in categorical_columns]
})

# Print the summary DataFrame
summary_df

,Column,DataType,NumUniqueValues
0,TransactionMonth,object,23
1,Citizenship,object,4
2,LegalType,object,6
3,Title,object,5
4,Language,object,1
5,Bank,object,11
6,AccountType,object,3
7,MaritalStatus,object,3
8,Gender,object,3
9,Country,object,1


Key Insights from the Summary:

- Columns like `Language, Country, ItemType, StatutoryClass, and StatutoryRiskType` don't provide variability. 
- Drop these columns from the dataset.


- `Model` (411 unique values) could add complexity.
- Consider reducing cardinality by grouping or applying target encoding.

- Columns like AlarmImmobiliser, TrackingDevice, and NewVehicle have only 2 unique values.

- `TransactionMonth` should be converted to a date format.
- Extract useful features like year, month, and quarter if necessary.

- `CapitalOutstanding` should be numeric and can be used as a continuous feature.

In [5]:
columns_to_drop = ['Language', 'Country', 'ItemType', 'StatutoryClass', 'StatutoryRiskType']
df = df.drop(columns=columns_to_drop)


Convert the `CapitalOutstanding` to numeric format

In [6]:
# Convert 'CapitalOutstanding' to numeric, coerce errors to handle non-numeric values (e.g., empty strings or NaNs)
df['CapitalOutstanding'] = pd.to_numeric(df['CapitalOutstanding'], errors='coerce')

# Fill missing values if necessary, e.g., with 0 or the column's mean
df['CapitalOutstanding'] = df['CapitalOutstanding'].fillna(0)  # Or df['CapitalOutstanding'].fillna(df['CapitalOutstanding'].mean())

# Ensure it's a float type
df['CapitalOutstanding'] = df['CapitalOutstanding'].astype(float)

# Verify the conversion
print(df['CapitalOutstanding'].dtype)


float64


Convert boolean 'IsVATRegistered' to integer

In [25]:
# Convert boolean 'IsVATRegistered' to integer
df['IsVATRegistered'] = df['IsVATRegistered'].astype(int)

Convert the `TransactionMonth` to a datetime format and extract relevant features.

In [27]:
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])
df['TransactionYear'] = df['TransactionMonth'].dt.year
df['TransactionMonthOnly'] = df['TransactionMonth'].dt.month
df['TransactionQuarter'] = df['TransactionMonth'].dt.quarter
# Drop the original datetime column if it's no longer needed
df = df.drop(columns=['TransactionMonth'])


Check the Distribution of `Title, MaritalStatus, and Gender`

In [8]:
print(f'Gender Distribution:\n{df.Gender.value_counts()}')
print(f'Title Distribution:\n {df.Title.value_counts()}')
print(f'Marital Status Distribution:\n {df.MaritalStatus.value_counts()}')

Gender Distribution:
Gender
Not specified    950526
Male              42817
Female             6755
Name: count, dtype: int64
Title Distribution:
 Title
Mr      933555
Mrs      45850
Ms       13269
Miss      6614
Dr         810
Name: count, dtype: int64
Marital Status Distribution:
 MaritalStatus
Not specified    994467
Single             4254
Married            1377
Name: count, dtype: int64


**High Proportion of Unspecified Gender:**

A significant portion of the `Gender` values (950,526) is missing or unspecified.
- These unspecified gender values can be filled using the information from the `Title` column where available.

**Title-to-Gender Mapping:**

- Titles like `Mr, Mrs, Miss, and Ms `provide a clear indication of gender.
- Dr is ambiguous and may not specify gender directly.

**Approach to Impute Missing Genders Using Titles:**

`Mr → Male`

`Mrs, Miss, Ms → Female`

`Dr` → Leave as Not specified (since it’s ambiguous)

In [9]:
# Map Titles to Gender
title_to_gender_map = {
    'Mr': 'Male',
    'Mrs': 'Female',
    'Ms': 'Female',
    'Miss': 'Female',
    'Dr': None  # Leave as None since it could be either gender
}

# Fill missing Gender based on Title
df['Gender'] = df.apply(lambda row: title_to_gender_map.get(row['Title'], row['Gender']), axis=1)

# Display the updated Gender counts
print(df['Gender'].value_counts())


Gender
Male      933555
Female     65733
Name: count, dtype: int64


**Balanced Gender Distribution:**

The number of `male and female` entries is now significantly more balanced compared to the initial state where a large portion of the Gender values were `unspecified`.

Dropping the `MaritalStatus` column is a reasonable decision given the high proportion of unspecified values

In [10]:
# Drop the 'MaritalStatus' column from the DataFrame

df = df.drop(columns=['MaritalStatus'])


In [11]:
# Drop the specified columns from the DataFrame
df = df.drop(columns=['Unnamed: 0', 'UnderwrittenCoverID', 'PolicyID'])

**Encoding Strategy:**

- **Label Encoding:** For binary or low-cardinality features.
- **One-Hot Encoding:** For moderate cardinality features where the number of new columns remains manageable.
- **Target Encoding:** For high cardinality features to reduce the number of new features while capturing useful patterns.
- **Frequency Encoding:** Another option for high cardinality features, where you replace categories with their frequency in the dataset.

In [28]:
# Define cardinality thresholds
low_cardinality_threshold = 10
high_cardinality_threshold = 20

# Identify categorical columns
categorical_cols = df.select_dtypes(include='object').columns

# Separate features by cardinality
low_cardinality_features = [col for col in categorical_cols if df[col].nunique() <= low_cardinality_threshold]
moderate_cardinality_features = [col for col in categorical_cols if low_cardinality_threshold < df[col].nunique() <= high_cardinality_threshold]
high_cardinality_features = [col for col in categorical_cols if df[col].nunique() > high_cardinality_threshold]

# Label Encoding for low cardinality features
label_encoders = {}
for feature in low_cardinality_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# One-Hot Encoding for moderate cardinality features
one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)
for feature in moderate_cardinality_features:
    encoded = one_hot_encoder.fit_transform(df[[feature]])
    encoded_df = pd.DataFrame(encoded, columns=one_hot_encoder.get_feature_names_out([feature]))
    df = pd.concat([df, encoded_df], axis=1)
    df = df.drop(columns=[feature])

# Frequency Encoding for high cardinality features (Example)
for feature in high_cardinality_features:
    freq = df[feature].value_counts()
    df[feature + '_freq'] = df[feature].map(freq)
    df = df.drop(columns=[feature])



# Save or proceed with further analysis/modeling
# df.to_csv('preprocessed_data.csv', index=False)  # Uncomment to save the preprocessed data


In [29]:
# Verify the transformations
df.head()


,IsVATRegistered,Citizenship,LegalType,Title,AccountType,Gender,Province,PostalCode,mmcode,VehicleType,...,CoverGroup_Income Protector,CoverGroup_Motor Comprehensive,CoverGroup_Standalone passenger liability,CoverGroup_Third Party Only,CoverGroup_Trailer,SubCrestaZone_freq,make_freq,Model_freq,CoverCategory_freq,CoverType_freq
0,1,0,0,2,0,1,2,1459,44069150.0,4,...,0.0,0.0,0.0,0.0,0.0,42168,41940,180,97313,103763
1,1,0,0,2,0,1,2,1459,44069150.0,4,...,0.0,0.0,0.0,0.0,0.0,42168,41940,180,97313,103763
2,1,0,0,2,0,1,2,1459,44069150.0,4,...,0.0,0.0,0.0,0.0,0.0,42168,41940,180,97313,103763
3,1,0,0,2,0,1,2,1459,44069150.0,4,...,0.0,0.0,0.0,0.0,0.0,42168,41940,180,8776,104183
4,1,0,0,2,0,1,2,1459,44069150.0,4,...,0.0,0.0,0.0,0.0,0.0,42168,41940,180,8776,104183


In [31]:
# Drop Columns if Missing values
df.dropna()

IsVATRegistered                                                                        0
Citizenship                                                                            0
LegalType                                                                              0
Title                                                                                  0
AccountType                                                                            0
Gender                                                                                 0
Province                                                                               0
PostalCode                                                                             0
mmcode                                                                                 0
VehicleType                                                                            0
RegistrationYear                                                                       0
Cylinders            

**Feature Selection and Dimensionality Reduction**

In [30]:

#  Feature Selection using SelectKBest
X = df.drop(columns=['TotalPremium', 'TotalClaims'])
y = df['TotalPremium']  # or 'TotalClaims' depending on the target

selector = SelectKBest(score_func=f_classif, k=20)  # Select top 20 features
X_new = selector.fit_transform(X, y)

# Example: Dimensionality Reduction using PCA
pca = PCA(n_components=20)  # Reduce to 20 principal components
X_pca = pca.fit_transform(X)

# Plot explained variance
plt.figure(figsize=(10, 6))
sns.lineplot(x=range(1, 21), y=np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Explained Variance')
plt.show()


ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [24]:

pd.set_option('display.max_rows', 200)
df.dtypes.reset_index()

,index,0
0,TransactionMonth,datetime64[ns]
1,IsVATRegistered,bool
2,Citizenship,int64
3,LegalType,int64
4,Title,int64
5,AccountType,int64
6,Gender,int64
7,Province,int64
8,PostalCode,int64
9,mmcode,float64


In [ ]:
sample_data.to_csv('../data/sample_data.csv')